## Signal Synchronization and Cropping

This script aligns and synchronizes kinematic landmarks, EMG signals, and video data for experimental analysis. It extracts timestamps, generates time vectors, crops and interpolates data streams, and synchronizes datasets across modalities. The script also visualizes the synchronization process and extracts relevant video segments for aligned intervals which are used later on in the Data analysis pipeline to crop the videos further into different task phases.

In [ ]:
import os
import glob
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d

# --------------------------------------------------------------------
# Global Settings
# --------------------------------------------------------------------
BASE_DIR = r"C:\Users\schmi\Documents\Studium\TUM\5. Semester\Masterthesis\Experimental Data"
PARTICIPANTS = list(range(1, 9))


KINEMATIC_SUBDIR = os.path.join("Kinematic Landmarks", "3D Landmarks") 
MYO_SUBDIR       = os.path.join("Processed EMG Data", "Processed Myo")
OTB_SUBDIR       = os.path.join("Processed EMG Data", "Processed OTB+")
SYNC_FOLDERNAME  = "Synchronized Data"  

SAMPLE_RATES = {
    'kinematic': 31.2,   # For both hand and pose
    'myo':       375,
    'otb':       2000
}

TIME_WINDOW = 120  # seconds search area for matching data streams

# --------------------------------------------------------------------
# 1) Timestamp Extraction 
# --------------------------------------------------------------------
def parse_kin_filename(time_str_npy):
    """
    Helper to parse time strings from filenames.
    Examples:
        '11_37_57.592' => 11*3600 + 37*60 + 57.592
        '11_37_57.592_3D' => 11*3600 + 37*60 + 57.592
    """
    time_str = time_str_npy.replace(".npy", "")
    parts = time_str.split("_")
    if len(parts) < 3:
        return None
    hh, mm, ss_milli = parts[0], parts[1], parts[2]
    milli = 0
    if '.' in ss_milli:
        ss_part, milli_part = ss_milli.split('.')
        try:
            milli = int(milli_part.ljust(3, '0')[:3])
        except ValueError:
            return None
    else:
        ss_part = ss_milli
    try:
        hours   = int(hh)
        minutes = int(mm)
        seconds = int(ss_part)
    except ValueError:
        return None
    return hours*3600 + minutes*60 + seconds + milli/1000

def extract_timestamp_hand(filename):
    if not filename.startswith("hand_landmarks_"):
        return None
    # Remove the prefix and the '_3D.npy' suffix
    time_str = filename[len("hand_landmarks_"):].replace("_3D.npy", "")
    return parse_kin_filename(time_str)

def extract_timestamp_pose(filename):
    if not filename.startswith("pose_landmarks_"):
        return None
    # Remove the prefix and the '_3D.npy' suffix
    time_str = filename[len("pose_landmarks_"):].replace("_3D.npy", "")
    return parse_kin_filename(time_str)


def extract_timestamp_myo(filename):
    """
    Expects: processed_myo_YYYY-MM-DD_10-43-32.688_normalized_envelope.npy
    => parse "10-43-32.688" => total secs from midnight
    """
    base = os.path.basename(filename)
    parts = base.split("_")
    if len(parts) < 4:
        return None
    time_part = parts[3]
    time_part = time_part.split("_")[0]
    if '-' not in time_part:
        return None
    hhmmss = time_part.split('-')
    if len(hhmmss) != 3:
        return None
    hh, mm, ss_milli = hhmmss
    milli = 0
    if '.' in ss_milli:
        ss_part, milli_part = ss_milli.split('.')
        try:
            milli = int(milli_part.ljust(3, '0')[:3])
        except ValueError:
            return None
    else:
        ss_part = ss_milli
    try:
        hours   = int(hh)
        minutes = int(mm)
        seconds = int(ss_part)
    except ValueError:
        return None
    return hours*3600 + minutes*60 + seconds + milli/1000

def extract_timestamp_otb(filename):
    """
    Expects: processed_emg_data_YYYYMMDD_104327483_normalized_emg.npy
    => parse "104327483" => total secs from midnight
    """
    base = os.path.basename(filename)
    parts = base.split("_")
    if len(parts) < 5:
        return None
    time_str = parts[4]  # e.g. "104327483"
    if len(time_str) < 6:
        return None
    try:
        hh = int(time_str[0:2])
        mm = int(time_str[2:4])
        ss = int(time_str[4:6])
        milli_str = time_str[6:]
        if not milli_str:
            milli = 0
        else:
            milli = int(milli_str.ljust(3, '0')[:3])
    except ValueError:
        return None
    return hh*3600 + mm*60 + ss + milli/1000.0

# --------------------------------------------------------------------
# 2) Time Vector Generation
# --------------------------------------------------------------------
def build_time_vector(start_sec, n_samples, sr):
    """
    length = n_samples => [start_sec + 0/sr, ..., start_sec + (n_samples-1)/sr]
    """
    return np.round(start_sec + np.arange(n_samples)/sr, 6)

# --------------------------------------------------------------------
# 3) Crop & Interpolate to match Sample Rates
# --------------------------------------------------------------------
def crop_data(time_vec, data_arr, start_t, end_t):
    idx_start = np.searchsorted(time_vec, start_t, side='left')
    idx_end   = np.searchsorted(time_vec, end_t, side='right')
    idx_start = max(0, idx_start)
    idx_end   = min(len(time_vec), idx_end)
    return time_vec[idx_start:idx_end], data_arr[idx_start:idx_end]

def interpolate_data(old_time, old_data, new_time):
    if old_time.size < 2 or old_data.shape[0] < 2:
        num_feats = old_data.shape[1] if old_data.ndim > 1 else 1
        return np.zeros((len(new_time), num_feats))
    if old_time.size != old_data.shape[0]:
        print("[WARN] Time vs data mismatch!", old_time.size, old_data.shape)
        num_feats = old_data.shape[1] if old_data.ndim > 1 else 1
        return np.zeros((len(new_time), num_feats))
    if old_data.ndim == 1:
        old_data = old_data.reshape(-1,1)
    num_features = old_data.shape[1]
    new_data = np.zeros((len(new_time), num_features))
    for i in range(num_features):
        f = interp1d(old_time, old_data[:, i], 
                     kind='linear', bounds_error=False,
                     fill_value='extrapolate')
        new_data[:, i] = f(new_time)
    return new_data

# --------------------------------------------------------------------
# 4) Load & Reshape Hand / Pose
# --------------------------------------------------------------------
def load_hand(file_path, sr=31.2):
    """
    [frame_idx, landmark_idx, X, Y, Z]
    => (n_frames, n_landmarks*3)
    => time_vec => shape (n_frames,)
    """
    arr = np.load(file_path)
    filename = os.path.basename(file_path)
    start_ts = extract_timestamp_hand(filename)
    if start_ts is None:
        start_ts = 0.0

    # --- Updated Shape Check ---
    if arr.shape[1] != 5:
        print(f"[WARN] {filename} => not (N,5). skipping hand.")
        return np.array([]), np.array([[]]), start_ts

    frame_idxs = arr[:,0].astype(int)
    lmk_idxs   = arr[:,1].astype(int)  
    n_frames   = frame_idxs.max()+1
    n_landmarks= lmk_idxs.max()+1
    data_3d = np.zeros((n_frames,n_landmarks,3), dtype=np.float32)
    for row in arr:
        f = int(row[0])
        l = int(row[1])  
        x, y, z = row[2], row[3], row[4]  
        data_3d[f, l] = [x, y, z]
    hand_data = data_3d.reshape(n_frames, -1)
    hand_time = build_time_vector(start_ts, n_frames, sr)
    return hand_time, hand_data, start_ts

def load_pose(file_path, sr=31.2):
    """
    [frame_idx, RSx, RSy, RSz, REx, REy, REz]
    => (n_frames,6)
    => time_vec => shape(n_frames,)
    """
    arr = np.load(file_path)
    filename = os.path.basename(file_path)
    start_ts = extract_timestamp_pose(filename)
    if start_ts is None:
        start_ts = 0.0

    # --- Updated Shape Check ---
    if arr.shape[1] != 7:
        print(f"[WARN] {filename} => not (N,7). skipping pose.")
        return np.array([]), np.array([[]]), start_ts

    frame_idxs = arr[:,0].astype(int)
    n_frames   = frame_idxs.max()+1
    pose_data  = np.zeros((n_frames,6), dtype=np.float32)
    for row in arr:
        f = int(row[0])
        RSx, RSy, RSz = row[1], row[2], row[3]  
        REx, REy, REz = row[4], row[5], row[6] 
        pose_data[f] = [RSx, RSy, RSz, REx, REy, REz]
    pose_time = build_time_vector(start_ts, n_frames, sr)
    return pose_time, pose_data, start_ts

# --------------------------------------------------------------------
# 5) Debug Plot
# --------------------------------------------------------------------
def debug_cropping_and_plot_all_channels(
    hand_t, hand_d,
    pose_t, pose_d,
    myo_t,  myo_d,
    otb_t,  otb_d,
    global_start, global_end,
    out_plot_dir,
    plot_basename="plot_all_channels"
):
    """
    Plots *all channels* for each signal (Hand, Pose, Myo, OTB) in separate subplots,
    places legends on the right, and saves the figure to out_plot_dir.

    hand_d, pose_d, myo_d, otb_d: shape (N, C) or (N,) => will be forced to (N,C).
    The time axis in the plot is shifted so that 'global_start' => 0 for display.

    Subplot Titles & Y-Labels:
      - Hand Landmarks => "Position (px)"
      - Elbow & Shoulder Landmarks => "Position (px)"
      - Myo Normalized Envelope => "Amplitude [%]"
      - OTB+ Normalized Envelope => "Amplitude [%]"
    """

    print(f"\n--- Cropping from {global_start:.3f}s to {global_end:.3f}s ---")
    print(f"Hand: {len(hand_t)} samples, shape={hand_d.shape}")
    print(f"Pose: {len(pose_t)} samples, shape={pose_d.shape}")
    print(f"Myo:  {len(myo_t)} samples, shape={myo_d.shape}")
    print(f"OTB:  {len(otb_t)} samples, shape={otb_d.shape}")

    # Ensure output folder exists
    os.makedirs(out_plot_dir, exist_ok=True)

    # 1) Force each data array to 2D
    def ensure_2d(arr):
        arr = np.asarray(arr)
        if arr.ndim == 1:
            return arr.reshape(-1, 1)
        return arr

    hand_d = ensure_2d(hand_d)
    pose_d = ensure_2d(pose_d)
    myo_d  = ensure_2d(myo_d)
    otb_d  = ensure_2d(otb_d)

    # 2) Shift times so the plot starts at 0
    ht0 = hand_t - global_start
    pt0 = pose_t - global_start
    mt0 = myo_t  - global_start
    ot0 = otb_t  - global_start

    # ----------------------------------------------------
    # 3) Helper to generate short labels for each channel
    # ----------------------------------------------------
    def get_labels(signal_type, n_channels):
        """
        Returns a list of short labels for each channel based on the signal_type.
        Adjust muscle_map or other lists as needed.
        """
        if signal_type == 'hand':
            # Each landmark has 3 dims (X, Y, Z)
            # => channel c -> landmark index = c//3, dim = c%3
            labels = []
            dims = ["X","Y","Z"]
            for c in range(n_channels):
                lmk = c // 3
                d   = dims[c % 3]
                labels.append(f"H{lmk+1}{d}")  # e.g. H1X, H1Y, H1Z, H2X, ...
            return labels

        elif signal_type == 'pose':
            # Mapping Landmark Dimensions for Shoulder and Elbow
            if n_channels == 6:
                return ["ShX","ShY","ShZ","EbX","EbY","EbZ"]
            else:
                return [f"Pch{c}" for c in range(n_channels)]

        elif signal_type == 'myo':
            # Mapping Muscles to Myo Channel
            muscle_map = ["ECU","EDC","ECR","BIC","TRI"]
            labels = []
            for c in range(n_channels):
                if c < len(muscle_map):
                    labels.append(muscle_map[c])
                else:
                    labels.append(f"Myo{c}") 
            return labels

        else: 
            return [f"OTB{c+1}" for c in range(n_channels)]

    # ----------------------------------------------------
    # 4) Function to plot each set of channels
    # ----------------------------------------------------
    def plot_all_ch(ax, time_vec, data_2d, title_str, ylabel_str, signal_type):
        """Plot each channel of data_2d with a short label."""
        if data_2d.size == 0:
            ax.set_title(f"{title_str} (No Data)")
            return

        ax.set_title(title_str, fontsize=11)
        colors = plt.cm.jet(np.linspace(0, 1, data_2d.shape[1]))

        # get short labels for this signal type
        ch_labels = get_labels(signal_type, data_2d.shape[1])

        for ch in range(data_2d.shape[1]):
            ax.plot(time_vec, data_2d[:, ch],
                    color=colors[ch],
                    label=ch_labels[ch])

        ax.set_ylabel(ylabel_str, fontsize=10)
        # Place legend on the right side, aligned to the top of the axes
        ax.legend(
            loc='upper left',
            bbox_to_anchor=(1.02, 1.0),
            borderaxespad=0.,
            fontsize='small',
            ncol=9
        )

    # ----------------------------------------------------
    # 5) Create subplots and plot each signal
    # ----------------------------------------------------
    fig, axs = plt.subplots(4, 1, figsize=(12, 14), sharex=True)

    # HAND
    plot_all_ch(axs[0], ht0, hand_d,
                "Hand Landmarks", "Position (px)",
                signal_type="hand")

    # POSE
    plot_all_ch(axs[1], pt0, pose_d,
                "Elbow & Shoulder Landmarks", "Position (px)",
                signal_type="pose")

    # MYO
    plot_all_ch(axs[2], mt0, myo_d,
                "Myo Normalized Envelope", "Amplitude [%]",
                signal_type="myo")

    # OTB+
    plot_all_ch(axs[3], ot0, otb_d,
                "OTB+ Normalized Envelope", "Amplitude [%]",
                signal_type="otb")

    axs[3].set_xlabel("Time from Overlap Start (s)", fontsize=10)

    # ----------------------------------------------------
    # 6) Layout & Saving
    # ----------------------------------------------------
    plt.tight_layout(rect=[0, 0, 0.65, 1])  # widen right margin for legend
    save_path = os.path.join(out_plot_dir, f"{plot_basename}.png")
    plt.savefig(save_path, dpi=150, bbox_inches='tight')
    plt.close(fig)

    print(f"Plot saved to: {save_path}")

# --------------------------------------------------------------------
# 6) Video Cropping
# --------------------------------------------------------------------
def parse_time_from_string(time_str):
    """
    Helper: parse e.g. "10_42_26.955" => 10*3600 + 42*60 + 26.955
    Returns None on failure.
    """
    parts = time_str.split("_")
    if len(parts) < 3:
        return None
    hh, mm, ss_milli = parts[0], parts[1], parts[2]
    milli = 0
    if "." in ss_milli:
        ss_part, milli_part = ss_milli.split(".")
        try:
            milli = int(milli_part.ljust(3, '0')[:3])
        except ValueError:
            return None
    else:
        ss_part = ss_milli
    try:
        hours = int(hh)
        mins  = int(mm)
        secs  = int(ss_part)
    except ValueError:
        return None
    return hours * 3600 + mins * 60 + secs + milli / 1000.0

def parse_video_recording_timestamp(video_path):
    """
    Try to extract the recording's absolute start time (seconds from midnight)
    by checking the folder name 'recording_10_42_26.955' or
    'annotated_videos_10_42_26.955'.
    
    Returns the float timestamp from midnight, or None if not found.
    """
    video_dir = os.path.dirname(video_path) 
    parent_dir = os.path.dirname(video_dir)  

    # 1) Try to parse from "recording_10_42_26.955"
    recording_base = os.path.basename(parent_dir)
    if recording_base.startswith("recording_"):
        time_str = recording_base[len("recording_"):]
        ts = parse_time_from_string(time_str)
        if ts is not None:
            return ts

    # 2) Otherwise, try from "annotated_videos_10_42_26.955"
    annotated_base = os.path.basename(video_dir)
    if annotated_base.startswith("annotated_videos_"):
        time_str = annotated_base[len("annotated_videos_"):]
        ts = parse_time_from_string(time_str)
        if ts is not None:
            return ts

    return None 

def crop_video(in_path, start_s, end_s, out_path):
    """
    Crops the video in `in_path` from local time start_s to end_s (seconds),
    writing result to `out_path`.
    """
    cap = cv2.VideoCapture(in_path)
    if not cap.isOpened():
        print(f"[ERROR] Could not open video: {in_path}")
        return False

    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    print(f"[DEBUG] crop_video -> FPS={fps:.2f}, total_frames={total_frames}, "
          f"start_s={start_s:.3f}, end_s={end_s:.3f}")

    start_frame = int(round(start_s * fps))
    end_frame   = int(round(end_s   * fps))

    print(f"[DEBUG] crop_video -> start_frame={start_frame}, end_frame={end_frame}")

    # If out of range
    if start_frame < 0:
        start_frame = 0
    if end_frame > total_frames:
        end_frame = total_frames

    # If no overlap
    if end_frame <= start_frame:
        print(f"[WARN] No overlap: start_frame={start_frame}, end_frame={end_frame} => skip cropping.")
        cap.release()
        return False

    fourcc = cv2.VideoWriter_fourcc(*"XVID")
    out_vid = cv2.VideoWriter(out_path, fourcc, fps, (w, h))
    if not out_vid.isOpened():
        print(f"[ERROR] Could not open VideoWriter for output: {out_path}")
        cap.release()
        return False

    # Read frames and write only [start_frame, end_frame)
    frame_idx = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break  # out of frames

        if start_frame <= frame_idx < end_frame:
            out_vid.write(frame)
        elif frame_idx >= end_frame:
            break

        frame_idx += 1

    out_vid.release()
    cap.release()
    return True

def videoCropping(participant_dir, matches_info):
    cropped_vids_dir = os.path.join(participant_dir, "Synchronized Data", "Cropped Videos")
    os.makedirs(cropped_vids_dir, exist_ok=True)

    # 1) Collect all .avi
    pattern = os.path.join(
        participant_dir,
        "Camera Data Timestamped",
        "recording_sessions",
        "**",
        "annotated_videos_*",
        "annotated_video_camera_0.avi"
    )
    all_video_paths = glob.glob(pattern, recursive=True)
    if not all_video_paths:
        print(f"[WARN] No annotated videos found under {participant_dir}.")
        return

    # 2) Build list of { 'path': ..., 'video_start_ts': ... }
    video_entries = []
    for vp in all_video_paths:
        # --- Skip calibration: 
        if "calibration" in vp.lower():
            # This path is for calibration – skip
            continue

        vts = parse_video_recording_timestamp(vp)
        if vts is not None:
            video_entries.append({'path': vp, 'video_start_ts': vts})

    # Sort by the absolute start time of the video
    video_entries.sort(key=lambda x: x['video_start_ts'])

    # Sort matches by the match timestamp
    matches_sorted = sorted(matches_info, key=lambda m: m['timestamp_sec'])

    # Pair Matches 1:1 in chronological order:
    n_videos = len(video_entries)
    n_matches = len(matches_sorted)
    n_to_crop = min(n_videos, n_matches)

    print(f"[INFO] Found {n_matches} matches and {n_videos} videos. Will crop {n_to_crop} videos.\n")

    for i in range(n_to_crop):
        match_i = matches_sorted[i]
        video_i = video_entries[i]

        match_num    = match_i["match_number"]
        global_start = match_i["global_start"]
        global_end   = match_i["global_end"]
        vid_path     = video_i["path"]
        vid_start_ts = video_i["video_start_ts"]

        # Convert from absolute times to local times
        local_start = global_start - vid_start_ts
        local_end   = global_end   - vid_start_ts

        out_vid_path = os.path.join(
            cropped_vids_dir,
            f"match_{match_num:02d}_cropped_camera_0.avi"
        )

        print(f"[INFO] Cropping video for Match #{match_num}:")
        print(f"       Input : {vid_path}")
        print(f"       Output: {out_vid_path}")
        print(f"       Global Start: {global_start:.3f}s, Global End: {global_end:.3f}s")
        print(f"       Video Start : {vid_start_ts:.3f}s => Local Start: {local_start:.3f}s, Local End: {local_end:.3f}s")

        success = crop_video(
            in_path=vid_path,
            start_s=local_start,
            end_s=local_end,
            out_path=out_vid_path
        )

        if success:
            print("      => Cropping succeeded.\n")
        else:
            print("      => Cropping failed or was skipped.\n")

# --------------------------------------------------------------------
# 7) Main
# --------------------------------------------------------------------
def main():
    TIME_WINDOW = 120  # seconds
    MAX_USE = 2        # Maximum number of times a file can be reused

    # For each participant separately, create a list that tracks all match info
    for p in PARTICIPANTS:
        participant_str = f"P({p})"
        participant_dir = os.path.join(BASE_DIR, participant_str)
        if not os.path.isdir(participant_dir):
            print(f"No folder for {participant_str}, skipping.")
            continue

        # -------------------------------
        # 1) Build dicts for Hand & Pose
        # -------------------------------
        hand_dict = {}
        pose_dict = {}
        kin_dir = os.path.join(participant_dir, KINEMATIC_SUBDIR)
        all_kin = glob.glob(os.path.join(kin_dir, "*.npy"))
        for fp in all_kin:
            fn = os.path.basename(fp)
            if fn.startswith("hand_landmarks_"):
                suffix = fn[len("hand_landmarks_"):]
                hand_dict[suffix] = fp
            elif fn.startswith("pose_landmarks_"):
                suffix = fn[len("pose_landmarks_"):]
                pose_dict[suffix] = fp

        # -------------------------------
        # 2) Gather Myo and OTB+ files
        # -------------------------------
        myo_files = []
        pat_myo = os.path.join(participant_dir, MYO_SUBDIR, "processed_myo_*_normalized_envelope.npy")
        for f in glob.glob(pat_myo):
            ts = extract_timestamp_myo(os.path.basename(f))
            if ts is not None:
                myo_files.append({'path': f, 'timestamp': ts})

        otb_files = []
        pat_otb = os.path.join(participant_dir, OTB_SUBDIR, "processed_emg_data_*_normalized_emg.npy")
        for f in glob.glob(pat_otb):
            ts = extract_timestamp_otb(os.path.basename(f))
            if ts is not None:
                otb_files.append({'path': f, 'timestamp': ts})

        myo_files.sort(key=lambda x: x['timestamp'])
        otb_files.sort(key=lambda x: x['timestamp'])

        # If no Myo or OTB+ files, skip
        if not myo_files or not otb_files:
            print(f"Skipping {participant_str}, missing Myo or OTB.")
            continue

        print(f"\n=== {participant_str}: Found {len(hand_dict)} hand, {len(pose_dict)} pose, "
              f"{len(myo_files)} Myo, {len(otb_files)} OTB ===")

        # -------------------------------
        # 3) Prepare output directories
        # -------------------------------
        out_dir = os.path.join(participant_dir, SYNC_FOLDERNAME)
        os.makedirs(out_dir, exist_ok=True)

        plot_dir = os.path.join(out_dir, "Plots of Synchronized Data")
        os.makedirs(plot_dir, exist_ok=True)

        # -------------------------------
        # 4) Usage Tracking & Sync Loop
        # -------------------------------
        used_myo = {}  # key: myo_idx, value: usage_count
        used_otb = {}  # key: otb_idx, value: usage_count
        sync_count = 0

        # This list will hold all the matches for current participant
        all_matches = []

        all_suffixes = sorted(set(hand_dict.keys()) | set(pose_dict.keys()))
        for suffix in all_suffixes:
            # 4.1) Load Hand / Pose for this suffix
            hand_time = np.array([])
            hand_data = np.array([[]])
            hts = None

            if suffix in hand_dict:
                htime, hdata, hts_ = load_hand(hand_dict[suffix], sr=SAMPLE_RATES['kinematic'])
                if htime.size >= 2 and hdata.shape[0] >= 2:
                    hand_time = htime
                    hand_data = hdata
                    hts = hts_
                else:
                    print(f"[WARN] Hand data for suffix {suffix} is insufficient.")

            pose_time = np.array([])
            pose_data = np.array([[]])
            pts = None

            if suffix in pose_dict:
                ptime, pdata, pts_ = load_pose(pose_dict[suffix], sr=SAMPLE_RATES['kinematic'])
                if ptime.size >= 2 and pdata.shape[0] >= 2:
                    pose_time = ptime
                    pose_data = pdata
                    pts = pts_
                else:
                    print(f"[WARN] Pose data for suffix {suffix} is insufficient.")

            # Skip if neither hand nor pose has enough data
            if hand_time.size < 2 and pose_time.size < 2:
                continue

            # 4.2) Determine a base k_ts from hand & pose
            if hts is not None and pts is not None:
                k_ts = min(hts, pts)  # or max(hts, pts)
            elif hts is not None:
                k_ts = hts
            elif pts is not None:
                k_ts = pts
            else:
                k_ts = 0.0

            # 4.3) Build candidate Myo files within TIME_WINDOW
            myo_candidates = [
                (abs(m['timestamp'] - k_ts), idx, m)
                for idx, m in enumerate(myo_files)
                if abs(m['timestamp'] - k_ts) <= TIME_WINDOW and used_myo.get(idx, 0) < MAX_USE
            ]
            myo_candidates.sort(key=lambda x: x[0])  # nearest first

            if not myo_candidates:
                # No suitable Myo => skip suffix
                continue

            # 4.4) Try each Myo candidate until OTB+ match is found
            pair_found = False
            for myo_diff, myo_idx, myo_f in myo_candidates:

                # 4.4.1) Build candidate OTB+ files
                otb_candidates = [
                    (abs(o['timestamp'] - k_ts), oidx, o)
                    for oidx, o in enumerate(otb_files)
                    if abs(o['timestamp'] - k_ts) <= TIME_WINDOW and used_otb.get(oidx, 0) < MAX_USE
                ]
                otb_candidates.sort(key=lambda x: x[0])

                if not otb_candidates:
                    
                    continue

                # 4.4.2) Use the nearest OTB candidate
                otb_diff, otb_idx, otb_f = otb_candidates[0]

                # 4.5) Load Myo & OTB
                print(f"\nSuffix {suffix}:")
                print(f"Selected Myo file: {myo_f['path']} (Timestamp: {myo_f['timestamp']} sec)")
                print(f"Selected OTB+ file: {otb_f['path']} (Timestamp: {otb_f['timestamp']} sec)")

                myo_arr = np.load(myo_f['path'])
                myo_time = build_time_vector(myo_f['timestamp'], len(myo_arr), SAMPLE_RATES['myo'])

                otb_arr = np.load(otb_f['path'])
                if (otb_arr.ndim == 2) and (otb_arr.shape[0] < otb_arr.shape[1]):
                    otb_arr = otb_arr.T
                otb_time = build_time_vector(otb_f['timestamp'], len(otb_arr), SAMPLE_RATES['otb'])
                print("\n--- Debug: Data Stream Info BEFORE Cropping ---")

                print(f"Hand: start={hand_time[0]:.3f}, end={hand_time[-1]:.3f}, total_samples={len(hand_time)}")
                print(f"Pose: start={pose_time[0]:.3f}, end={pose_time[-1]:.3f}, total_samples={len(pose_time)}")
                print(f"Myo : start={myo_time[0]:.3f}, end={myo_time[-1]:.3f}, total_samples={len(myo_time)}")
                print(f"OTB : start={otb_time[0]:.3f}, end={otb_time[-1]:.3f}, total_samples={len(otb_time)}")

                # 4.6) Compute overlap
                def get_minmax(tv):
                    return (tv[0], tv[-1]) if len(tv) >= 2 else (np.inf, -np.inf)

                hstart, hend = get_minmax(hand_time)
                pstart, pend = get_minmax(pose_time)
                mstart, mend = get_minmax(myo_time)
                ostart, oend = get_minmax(otb_time)

                global_start = max(hstart, pstart, mstart, ostart)
                global_end   = min(hend, pend, mend, oend)
                print("\n--- Debug: Overlap Computation ---")
                print(f"global_start={global_start:.3f}, global_end={global_end:.3f}, "
                    f"overlap_duration={(global_end - global_start):.3f}")

                if global_end <= global_start:
                    print("No overlap => skip this Myo-OTB+ pair.")
                    continue

                # 4.7) Crop
                h_t_crop, h_d_crop = crop_data(hand_time, hand_data, global_start, global_end)
                p_t_crop, p_d_crop = crop_data(pose_time, pose_data, global_start, global_end)
                m_t_crop, m_d_crop = crop_data(myo_time,  myo_arr,   global_start, global_end)
                o_t_crop, o_d_crop = crop_data(otb_time,  otb_arr,   global_start, global_end)
                

                # 4.8) Interpolate to OTB+ reference
                ref_time  = o_t_crop
                hand_sync = interpolate_data(h_t_crop, h_d_crop, ref_time)
                pose_sync = interpolate_data(p_t_crop, p_d_crop, ref_time)
                myo_sync  = interpolate_data(m_t_crop, m_d_crop, ref_time)
                otb_sync  = o_d_crop  # no interpolation needed

                # 4.9) Plot
                plot_basename = f"{suffix}_all_channels"
                debug_cropping_and_plot_all_channels(
                    ref_time,   hand_sync,
                    ref_time,   pose_sync,
                    ref_time,   myo_sync,
                    ref_time,   otb_sync,
                    global_start, global_end,
                    out_plot_dir=plot_dir,
                    plot_basename=plot_basename
                )

                if len(ref_time) < 2:
                    print("OTB+ data too short => skip")
                    continue

                # 4.10) Combine
                if (hand_sync.shape[1] == 0) and (pose_sync.shape[1] == 0):
                    print("No hand or pose => skip.")
                    continue
                elif hand_sync.shape[1] == 0:
                    combined_kin = pose_sync
                elif pose_sync.shape[1] == 0:
                    combined_kin = hand_sync
                else:
                    combined_kin = np.hstack([hand_sync, pose_sync])

                # 4.11) Save
                sync_count += 1
                base_out = f"match_{sync_count:02d}"

                final_time = ref_time - ref_time[0]
                np.save(os.path.join(out_dir, f"{base_out}_time.npy"), final_time)
                np.save(os.path.join(out_dir, f"{base_out}_kin.npy"),  combined_kin)
                np.save(os.path.join(out_dir, f"{base_out}_myo.npy"),  myo_sync)
                np.save(os.path.join(out_dir, f"{base_out}_otb.npy"),  otb_sync)

                used_myo[myo_idx] = used_myo.get(myo_idx, 0) + 1
                used_otb[otb_idx] = used_otb.get(otb_idx, 0) + 1

                duration = final_time[-1] if len(final_time) > 0 else 0
                print(f"Match {sync_count:02d}: Overlap => {len(ref_time)} samples, duration ~{duration:.3f}s")

                #Parse the suffix to get absolute time from midnight
                ts_kin = parse_kin_filename(suffix.replace(".npy", ""))
                if ts_kin is None:
                    ts_kin = 0.0

                # Save the match info for video cropping
                all_matches.append({
                    "match_number": sync_count,
                    "suffix": suffix,
                    "timestamp_sec": ts_kin,    # parsed from e.g. "10_42_26.955"
                    "global_start": global_start, # e.g. 38546.955
                    "global_end":   global_end
                })

                pair_found = True
                break  

            if not pair_found:
                print(f"No valid Myo-OTB+ pair found within {TIME_WINDOW}s for suffix {suffix}. Skipping.")

        # Done with all suffixes
        if sync_count == 0:
            print(f"No successful overlap matches for {participant_str}.")
        else:
            print(f"Total successful matches for {participant_str}: {sync_count}")
            # After matching, crop the videos
            videoCropping(participant_dir, all_matches)

if __name__=="__main__":
    main()



=== P(5): Found 24 hand, 24 pose, 24 Myo, 24 OTB ===

Suffix 12_28_54.090_3D.npy:
Selected Myo file: C:\Users\schmi\Documents\Studium\TUM\5. Semester\Masterthesis\Experimental Data\P(5)\Processed EMG Data\Processed Myo\processed_myo_2024-10-31_12-28-41.259_normalized_envelope.npy (Timestamp: 44921.259 sec)
Selected OTB+ file: C:\Users\schmi\Documents\Studium\TUM\5. Semester\Masterthesis\Experimental Data\P(5)\Processed EMG Data\Processed OTB+\processed_emg_data_20241031_122849737_normalized_emg.npy (Timestamp: 44929.737 sec)

--- Debug: Data Stream Info BEFORE Cropping ---
Hand: start=44934.090, end=44949.891, total_samples=494
Pose: start=44934.090, end=44949.891, total_samples=494
Myo : start=44921.259, end=44955.643, total_samples=12895
OTB : start=44929.737, end=44954.537, total_samples=49600

--- Debug: Overlap Computation ---
global_start=44934.090, global_end=44949.891, overlap_duration=15.801

--- Cropping from 44934.090s to 44949.891s ---
Hand: 31603 samples, shape=(31603, 63

C:\Users\schmi\AppData\Local\Temp\ipykernel_12020\1952413981.py:390: UserWarning: Tight layout not applied. The left and right margins cannot be made large enough to accommodate all axes decorations.
  plt.tight_layout(rect=[0, 0, 0.65, 1])  # widen right margin for legend


Plot saved to: C:\Users\schmi\Documents\Studium\TUM\5. Semester\Masterthesis\Experimental Data\P(5)\Synchronized Data\Plots of Synchronized Data\12_28_54.090_3D.npy_all_channels.png
Match 01: Overlap => 31603 samples, duration ~15.801s

Suffix 12_29_47.627_3D.npy:
Selected Myo file: C:\Users\schmi\Documents\Studium\TUM\5. Semester\Masterthesis\Experimental Data\P(5)\Processed EMG Data\Processed Myo\processed_myo_2024-10-31_12-29-31.393_normalized_envelope.npy (Timestamp: 44971.393 sec)
Selected OTB+ file: C:\Users\schmi\Documents\Studium\TUM\5. Semester\Masterthesis\Experimental Data\P(5)\Processed EMG Data\Processed OTB+\processed_emg_data_20241031_122944440_normalized_emg.npy (Timestamp: 44984.44 sec)

--- Debug: Data Stream Info BEFORE Cropping ---
Hand: start=44987.627, end=45002.306, total_samples=459
Pose: start=44987.627, end=45002.306, total_samples=459
Myo : start=44971.393, end=45017.836, total_samples=17417
OTB : start=44984.440, end=45007.240, total_samples=45600

--- Debug